<a href="https://colab.research.google.com/github/likhon15-4915/Fraud-Detection-DMML/blob/main/Fraud_Detection_RF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
data=pd.read_csv('/content/drive/MyDrive/Fraud Detection/Synthetic_Financial_datasets_log.csv')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
data

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,1,PAYMENT,9839.64,C1231006815,170136.00,160296.36,M1979787155,0.00,0.00,0,0
1,1,PAYMENT,1864.28,C1666544295,21249.00,19384.72,M2044282225,0.00,0.00,0,0
2,1,TRANSFER,181.00,C1305486145,181.00,0.00,C553264065,0.00,0.00,1,0
3,1,CASH_OUT,181.00,C840083671,181.00,0.00,C38997010,21182.00,0.00,1,0
4,1,PAYMENT,11668.14,C2048537720,41554.00,29885.86,M1230701703,0.00,0.00,0,0
...,...,...,...,...,...,...,...,...,...,...,...
6362615,743,CASH_OUT,339682.13,C786484425,339682.13,0.00,C776919290,0.00,339682.13,1,0
6362616,743,TRANSFER,6311409.28,C1529008245,6311409.28,0.00,C1881841831,0.00,0.00,1,0
6362617,743,CASH_OUT,6311409.28,C1162922333,6311409.28,0.00,C1365125890,68488.84,6379898.11,1,0
6362618,743,TRANSFER,850002.52,C1685995037,850002.52,0.00,C2080388513,0.00,0.00,1,0


In [ ]:
data.isna().sum()

,0
step,0
type,0
amount,0
nameOrig,0
oldbalanceOrg,0
newbalanceOrig,0
nameDest,0
oldbalanceDest,0
newbalanceDest,0
isFraud,0


In [ ]:

data['type'] = data['type'].map({'PAYMENT': 0, 'TRANSFER': 1, 'CASH_OUT': 2, 'DEBIT': 3})
data['isFraud'] = data['isFraud'].map({'No Fraud':0,'Fraud':1})

In [ ]:
data = data.drop('nameOrig', axis=1)


In [ ]:
data = data.drop('nameDest', axis=1)

In [ ]:
data

,step,type,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,1,0.0,9839.64,170136.00,160296.36,0.00,0.00,NaN,0
1,1,0.0,1864.28,21249.00,19384.72,0.00,0.00,NaN,0
2,1,1.0,181.00,181.00,0.00,0.00,0.00,NaN,0
3,1,2.0,181.00,181.00,0.00,21182.00,0.00,NaN,0
4,1,0.0,11668.14,41554.00,29885.86,0.00,0.00,NaN,0
...,...,...,...,...,...,...,...,...,...
6362615,743,2.0,339682.13,339682.13,0.00,0.00,339682.13,NaN,0
6362616,743,1.0,6311409.28,6311409.28,0.00,0.00,0.00,NaN,0
6362617,743,2.0,6311409.28,6311409.28,0.00,68488.84,6379898.11,NaN,0
6362618,743,1.0,850002.52,850002.52,0.00,0.00,0.00,NaN,0


In [ ]:
data = data.drop('isFraud', axis=1)

In [ ]:
x = data.drop('isFlaggedFraud', axis=1)
y = data[["isFlaggedFraud"]]

In [ ]:
x,y

(         step  type      amount  oldbalanceOrg  newbalanceOrig  \
 0           1   0.0     9839.64      170136.00       160296.36   
 1           1   0.0     1864.28       21249.00        19384.72   
 2           1   1.0      181.00         181.00            0.00   
 3           1   2.0      181.00         181.00            0.00   
 4           1   0.0    11668.14       41554.00        29885.86   
 ...       ...   ...         ...            ...             ...   
 6362615   743   2.0   339682.13      339682.13            0.00   
 6362616   743   1.0  6311409.28     6311409.28            0.00   
 6362617   743   2.0  6311409.28     6311409.28            0.00   
 6362618   743   1.0   850002.52      850002.52            0.00   
 6362619   743   2.0   850002.52      850002.52            0.00   
 
          oldbalanceDest  newbalanceDest  
 0                  0.00            0.00  
 1                  0.00            0.00  
 2                  0.00            0.00  
 3              21182.

In [ ]:
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
min = MinMaxScaler()
x = min.fit_transform(x)
lb = LabelEncoder()
y = lb.fit_transform(y)

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [ ]:
from sklearn.model_selection import train_test_split

xtrain,xtest,ytrain,ytest = train_test_split(x,y, test_size=0.2)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier()
model.fit(xtrain, ytrain)
model.score(xtest, ytest)


0.9999984283204089

In [ ]:
step = int(input())
type = int(input())
amount = int(input())
oldbalanceOrg =int(input())
newbalanceOrig= int(input())
oldbalanceDest = int(input())
newbalanceDest = int(input())

1
2
3
4
5
6
7


In [ ]:
values = [[step, type, amount,oldbalanceOrg, newbalanceOrig, oldbalanceDest,newbalanceDest]]

In [ ]:
values = min.transform(values)

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


In [ ]:
values

array([[0.00000000e+00, 6.66666667e-01, 3.24515467e-08, 6.71309439e-08,
        1.00836865e-07, 1.68531804e-08, 1.96530242e-08]])

In [ ]:
predicted_class = model.predict(values)

In [ ]:
lb.inverse_transform(predicted_class)[0]

0

In [ ]:
import joblib
joblib.dump(model, "randomforest.joblib")

['randomforest.joblib']

In [ ]:
joblib.dump(lb, "labelencoder.joblib")
joblib.dump(min, "minmaxscalar.joblib")

['minmaxscalar.joblib']